In [ ]:
import matplotlib
matplotlib.use("Agg")

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
%matplotlib inline
mpl_cols = ['#3388dd', '#aa3377', '#449911']

In [ ]:
import os, sys, inspect
# realpath() will make your script run, even if you symlink it :)
cmd_folder = os.path.realpath(os.path.abspath(
        os.path.join(os.path.split(inspect.getfile( inspect.currentframe() ))[0],
                     '..', '..', 'labeled-experiments', 'nn-classifiers')))
print 'Added {0} to path.'.format(cmd_folder)
if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)

In [ ]:
import labeling_network as lbln

from matplotlib import pyplot as plt
import theano
import theano.tensor as T
import numpy as np
from labeling_network import FullyConnectedLayer, ConvPoolLayer

import time
import datetime

In [ ]:
rng = np.random.RandomState(12345678)

In [ ]:
dataPath = '../../../factory-robot-data/imgs_2015-10-18/'

In [ ]:
def load_action_data(filename, lower, upper):
    action_file = open(filename)
    lines = action_file.readlines()[lower:upper]
    action_file.close()
    data = np.asarray([int(l) for l in lines])
    return data
#     return data.reshape(upper - lower, n_direction_sensors * n_classes)


In [ ]:
def convert_actions(actions, n_actions):
    result = []
    for a in actions:
        x = [0.0] * n_actions
        x[a] = 1.0
        result.append(x)
    return np.asarray(result, dtype=theano.config.floatX)
        

In [ ]:
def construct_training_examples(label_data, action_data, delta_t, n_past, n_future):
    assert len(label_data) == len(action_data)
    percept_len = label_data.shape[1]
    n_actions = action_data.shape[1]
    
    xs_length = n_past*percept_len + (n_past + n_future - 1)*n_actions
    t_length = n_future*percept_len
    
    training_data_xs = np.empty((len(label_data) - (n_past + n_future - 1)*delta_t, xs_length), 
                                dtype=theano.config.floatX)
    training_data_ts = np.empty((len(label_data) - (n_past + n_future - 1)*delta_t, t_length),
                                dtype=theano.config.floatX)
    
    for i in xrange(n_past * delta_t, len(label_data) - ((n_future-1)*delta_t + 1)):
        example = []
        for j in xrange(n_future):
            training_data_ts[i - n_past*delta_t, j*percept_len: (j+1)*percept_len] = np.asarray(
                label_data[i+j], dtype=theano.config.floatX)
            
            if (n_future - j) > 1:
                xs_a = np.mean(action_data[i + j*delta_t: 
                                           i + (j+1)*delta_t] , axis=0)
#                 print -(n_future-j-1)*n_actions
#                 print -(n_future-j-2)*n_actions
#                 print xs_a.shape
#                 print
                if n_future - j == 2:
                    training_data_xs[i - n_past*delta_t, -(n_future-j-1)*n_actions:] = xs_a
                else:
                    training_data_xs[i - n_past*delta_t, -(n_future-j-1)*n_actions:
                                                         -(n_future-j-2)*n_actions] = xs_a
        for j in xrange(n_past):
            xs_d = label_data[i - (n_past*delta_t) + j*delta_t]
            xs_a = np.mean(action_data[i - (n_past*delta_t) + j*delta_t : 
                                       i - (n_past*delta_t) + (j+1)*delta_t] , axis=0)
            training_data_xs[i - n_past*delta_t, j*(percept_len + n_actions):
                                                 (j+1)*percept_len + j*n_actions] = xs_d
            training_data_xs[i - n_past*delta_t, (j+1)*percept_len + j*n_actions:
                                                 (j+1)*(percept_len + n_actions)] = xs_a
             
    return training_data_xs, training_data_ts

In [ ]:
def shuffle_data(data, rng):
    xs, ts = data
    index_set = np.asarray(range(len(xs)))
    rng.shuffle(index_set)
    return xs[index_set], ts[index_set]

In [ ]:
n_train = 150000
n_valid = 5000
n_test = 1000

n_direction_sensors=9
n_classes=2
n_actions=3
delta_t = 2
n_past = 15
n_future = 12


lower = 0
upper = n_train + n_valid + n_test + delta_t*n_past



load_time_start = time.time()
label_data_raw = lbln.load_labeling_data(dataPath+'labels.dat', lower, upper, mask=-1, 
                                         n_direction_sensors=n_direction_sensors, 
                                         n_classes=n_classes)
actions_raw = convert_actions(load_action_data(dataPath+'actions.dat', lower, upper), 
                              n_actions=n_actions)

all_data = construct_training_examples(label_data_raw, actions_raw, delta_t, n_past, n_future)
all_data = shuffle_data(all_data, rng)

training_xs = theano.shared(all_data[0][:n_train], borrow=True)
training_ts = theano.shared(all_data[1][:n_train], borrow=True)

valid_xs = theano.shared(all_data[0][n_train: n_train+n_valid], borrow=True)
valid_ts = theano.shared(all_data[1][n_train: n_train+n_valid], borrow=True)

test_xs = theano.shared(all_data[0][n_train+n_valid:], borrow=True)
test_ts = theano.shared(all_data[1][n_train+n_valid:], borrow=True)


print 'Loading data took {0:.5} seconds'.format(time.time() - load_time_start)

In [ ]:
def visualize_frame(ax, percept, n_sensors, action, color=mpl_cols[0]):
    percept_length = len(percept)
    n_actions = len(action)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.bar(np.arange(percept_length), percept, 
           color=color,
           edgecolor='none',
           align='center')
    ax.bar(np.arange(percept_length, percept_length + n_actions), action,
          color=mpl_cols[1],
          edgecolor='none',
          align='center')
    ax.set_xlim(-0.5, percept_length+n_actions-0.5)
    ax.set_ylim(0, 1)
    ax.axvline(x=n_sensors-0.5)
    ax.axvline(x=percept_length-0.5)

def visualize_example(x, t, percept_length, n_sensors, n_actions, n_past, n_future):
    fig = plt.figure(figsize=(6, 1*(n_past+1)))
    for i in xrange(n_past):
        ax = fig.add_subplot(n_past + n_future, 1, i + 1)
        visualize_frame(ax, 
                        x[i*(percept_length + n_actions):
                             (i + 1)*percept_length + i*n_actions],
                        n_sensors,
                        x[(i+1)*percept_length + i*n_actions:
                          (i + 1)*(percept_length + n_actions)])
    for i in xrange(n_future-1):
        print x[n_past*(percept_length + n_actions) + i*n_actions:
                n_past*(percept_length + n_actions) + (i + 1)*n_actions]      
    
    for i in xrange(n_future):
        ax = fig.add_subplot(n_past + n_future, 1, n_past + i + 1)
        visualize_frame(ax, 
                        t[i*percept_length: (i+1)*percept_length], 
                        n_sensors, 
                        np.zeros(n_actions), 
                        color=mpl_cols[2])
    plt.show()

In [ ]:
for i in xrange(9, 10, delta_t):
    visualize_example(training_xs.get_value()[i], training_ts.get_value()[i], 
                      n_direction_sensors*n_classes, 
                      n_direction_sensors, 
                      n_actions, 
                      n_past, 
                      n_future)

##### Training parameters

In [ ]:
mini_batch_size = 10

### Train the Network. 

In [ ]:
network = lbln.Network([
        FullyConnectedLayer(n_in=n_past*(n_direction_sensors*n_classes + n_actions) + (n_future-1)*n_actions, 
                            n_out=2000),
        FullyConnectedLayer(n_in=2000, 
                            n_out=n_direction_sensors*n_classes*n_future)
    ], mini_batch_size)

In [ ]:
network.SGD((training_xs, training_ts), 
            100, 
            mini_batch_size, 
            0.2, 
            (valid_xs, valid_ts), 
            (test_xs, test_ts),
            'trained-networks/t_nf' + str(n_future) + '_np' + str(n_past) + '_' + datetime.datetime.now().strftime('%Y%m%d%H%M%S'),
#             learning_curve_file_name='decoder_learning_curve_bigdata',
#             rmsprop=(0.001, 0.9, 1e-6, 1.0)
            adam=(0.0005, 0.1, 0.001, 1e-8)
           )

### Test network.

In [ ]:
# network_filename = 'trained-networks/t_nf12_np15_20151019012110'

n_eval = 7000

simple_errors = []
net_errors = []


# net = lbln.Network.load_from_file(network_filename, mini_batch_size)
net = network
for x, t in zip(valid_xs.get_value()[:n_eval], valid_ts.get_value()[:n_eval]):
    net_prediction = net.get_single_output(x)
    net_errors.append(np.mean((net_prediction - t) ** 2))
print 'mse net:    {0}'.format(np.mean(net_errors))

In [ ]:
i = rng.randint(valid_xs.get_value().shape[0])
ex = valid_xs.get_value()[i]
visualize_example(ex, valid_ts.get_value()[i], 
                      n_direction_sensors*n_classes, 
                      n_direction_sensors, 
                      n_actions, 
                      n_past, 
                      n_future)

visualize_example(ex, net.get_single_output(ex), 
                      n_direction_sensors*n_classes, 
                      n_direction_sensors, 
                      n_actions, 
                      n_past, 
                      n_future)

## Visualize predictions

In [ ]:
# prediction_net_filename = 'trained-networks/t_nf12_np15_20151019012110'
visualization_net_filename = 'trained-networks/decoders/d_20151018193853'


# prediction_net = lbln.Network.load_from_file(prediction_net_filename, mini_batch_size)
prediction_net = network
visualization_net = lbln.Network.load_from_file(visualization_net_filename, mini_batch_size)

xs = valid_xs.get_value()
ts = valid_ts.get_value()

def extract_frames_from_x(x, n_direction_sensors, n_classes, n_actions, n_past):
    return np.asarray([x[ i   *(n_direction_sensors*n_classes + n_actions):
                         (i+1)*n_direction_sensors*n_classes + i*n_actions] for i in xrange(n_past)])
def extract_frames_from_t(t, n_direction_sensors, n_classes, n_future):
    return np.asarray([t[i*n_direction_sensors*n_classes:
                         (i+1)*n_direction_sensors*n_classes] for i in xrange(n_future)])

# print extract_frames_from_x(xs[0], n_direction_sensors, n_classes, n_actions, n_past)
# print extract_frames_from_t(ts[0], n_direction_sensors, n_classes, n_past)


def make_animation(filename, frames_past, frames_target, frames_predicted, visualization_net, n_direction_sensors):
    import matplotlib.animation as manimation

    FFMpegWriter = manimation.writers['ffmpeg']
    metadata = dict(title='anim', artist='an',
            comment='comment')
    writer = FFMpegWriter(fps=1, metadata=metadata)

    fig = plt.figure()
    
    def export_frame(fig, writer, frame_left, frame_right, n_direction_sensors, annotation=''):
        fig.clear()

        vis_l = (visualization_net.get_single_output(frame_left)).reshape(64, 64, 3)
        vis_l = matplotlib.colors.hsv_to_rgb(vis_l)
        
        vis_r = (visualization_net.get_single_output(frame_right)).reshape(64, 64, 3)
        vis_r = matplotlib.colors.hsv_to_rgb(vis_r)

        ax1 = fig.add_subplot(2, 2, 1)
        ax1.imshow(vis_l, interpolation='nearest')
        ax1.text(0.5, -0.1, annotation,
            verticalalignment='bottom', horizontalalignment='right',
            transform=ax1.transAxes,
            color='black', fontsize=7)


        ax2 = fig.add_subplot(2, 2, 2)
        ax2.imshow(vis_r, interpolation='nearest')
        ax2.text(0.5, -0.1, annotation,
            verticalalignment='bottom', horizontalalignment='right',
            transform=ax2.transAxes,
            color='black', fontsize=7)

        ax3 = fig.add_subplot(2, 2, 3)
        ax3.set_ylim(0, 1)
        for j in xrange(len(frame_left) / n_direction_sensors):
            ax3.bar(np.arange(n_direction_sensors) + j*n_direction_sensors,
                    frame_left[j*n_direction_sensors:(j+1)*n_direction_sensors],
                    color=mpl_cols[j % len(mpl_cols)],
                    edgecolor='none')


        ax4 = fig.add_subplot(2, 2, 4)
        ax4.set_ylim(0, 1)
        for j in xrange(len(frame_right) / n_direction_sensors):
            ax4.bar(np.arange(n_direction_sensors) + j*n_direction_sensors,
                    frame_right[j*n_direction_sensors:(j+1)*n_direction_sensors],
                    color=mpl_cols[j % len(mpl_cols)],
                    edgecolor='none')

        ax1.set_frame_on(False)
        ax1.get_yaxis().set_visible(False)
        ax1.get_xaxis().set_visible(False)
        ax2.set_frame_on(False)
        ax2.get_yaxis().set_visible(False)
        ax2.get_xaxis().set_visible(False)            
        ax3.get_xaxis().set_visible(False)
        ax4.get_yaxis().set_visible(False)
        ax4.get_xaxis().set_visible(False)
        writer.grab_frame()
    
    with writer.saving(fig, filename, 300):
        for i in xrange(len(frames_past)):
            export_frame(fig, writer, frames_past[i], frames_past[i], n_direction_sensors)
        for i in xrange(len(frames_target)):
            export_frame(fig, writer, frames_target[i], frames_predicted[i], n_direction_sensors, annotation='future')

            
            
for test_index in range(5, 15):         
    past_frames = extract_frames_from_x(xs[test_index], n_direction_sensors, n_classes, n_actions, n_past)
    target_frames = extract_frames_from_t(ts[test_index], n_direction_sensors, n_classes, n_future)
    predicted_frames = extract_frames_from_t(prediction_net.get_single_output(xs[test_index]), n_direction_sensors, n_classes, n_future)


    make_animation('anims/anim{0}.mp4'.format(test_index), 
                   past_frames, 
                   target_frames,
                   predicted_frames,  
                   visualization_net,
                   n_direction_sensors)

